In [ ]:
import pandas as pd

# Load the input files using Pandas
item_list_df = pd.read_excel('ITEM LIST.xlsx')
price_list_df = pd.read_excel('PRICE LIST.xlsx')
factory_demand_df = pd.read_excel('FACTORY DEMAND.xlsx')
target_volume_df = pd.read_excel('TARGET VOLUME.xlsx')

# Print data types to check if any conversion is needed
print("Item List Data Types:\n", item_list_df.dtypes)
print("Price List Data Types:\n", price_list_df.dtypes)
print("Factory Demand Data Types:\n", factory_demand_df.dtypes)
print("Target Volume Data Types:\n", target_volume_df.dtypes)

# Convert columns to appropriate data types if needed
item_list_df['PCX SPL MTL ID'] = item_list_df['PCX SPL MTL ID'].astype(str).fillna(item_list_df['PCX MTL ID'])
price_list_df['PCX SPL MTL ID'] = price_list_df['PCX SPL MTL ID'].astype(str).fillna(price_list_df['PCX MTL ID'])
factory_demand_df['PCX SPL MTL ID'] = factory_demand_df['PCX SPL MTL ID'].astype(str).fillna(factory_demand_df['PCX MTL ID'])

# Handle any potential conversion issues
factory_demand_df['VOLUME'] = pd.to_numeric(factory_demand_df['VOLUME'], errors='coerce').fillna(0).astype(int)
price_list_df['BID PRICE'] = pd.to_numeric(price_list_df['BID PRICE'], errors='coerce').fillna(0).astype(float)
target_volume_df['TOTAL VOLUME'] = pd.to_numeric(target_volume_df['TOTAL VOLUME'], errors='coerce').fillna(0).astype(int)
target_volume_df['TARGET %'] = pd.to_numeric(target_volume_df['TARGET %'], errors='coerce').fillna(0).astype(float)
target_volume_df['TARGET % RANGE'] = pd.to_numeric(target_volume_df['TARGET % RANGE'], errors='coerce').fillna(0).astype(float)

# Calculate MIN TARGET VOLUME and MAX TARGET VOLUME
target_volume_df['MIN TARGET VOLUME'] = (target_volume_df['TARGET %'] - target_volume_df['TARGET % RANGE']) * target_volume_df['TOTAL VOLUME']
target_volume_df['MAX TARGET VOLUME'] = (target_volume_df['TARGET %'] + target_volume_df['TARGET % RANGE']) * target_volume_df['TOTAL VOLUME']

print("Data Preprocessing Complete")

In [ ]:
# Merge ITEM LIST with PRICE LIST on 'PCX SPL MTL ID'
merged_df = pd.merge(item_list_df, price_list_df, on='PCX SPL MTL ID', how='inner')

# Assign preference by LOCAL FOR LOCAL and BID PRICE
merged_df['LOCAL_FOR_LOCAL'] = merged_df['SUPPLIER_x'] == merged_df['SUPPLIER COUNTRY_y']
merged_df = merged_df.sort_values(by=['LOCAL_FOR_LOCAL', 'BID PRICE'], ascending=[False, True])

merged_df.head()

In [ ]:
def allocate_volume(merged_df, target_volume_df, factory_demand_df):
    allocations = []
    
    for _, supplier in target_volume_df.iterrows():
        supplier_name = supplier['SUPPLIER']
        min_volume = supplier['MIN TARGET VOLUME']
        max_volume = supplier['MAX TARGET VOLUME']
        supplier_df = merged_df[merged_df['SUPPLIER_x'] == supplier_name]
        
        allocated_volume = 0
        for _, row in supplier_df.iterrows():
            if allocated_volume >= max_volume:
                break

            factory_options = factory_demand_df[factory_demand_df['PCX SPL MTL ID'] == row['PCX SPL MTL ID']]
            for _, factory in factory_options.iterrows():
                if allocated_volume >= max_volume:
                    break
                if factory['VOLUME'] > 0:
                    volume_to_allocate = min(max_volume - allocated_volume, factory['VOLUME'])
                    allocations.append({
                        'PCX SPL MTL ID': row['PCX SPL MTL ID'],
                        'SUPPLIER': row['SUPPLIER_x'],
                        'FACTORY CODE': factory['FACTORY CODE'],
                        'ALLOCATION (UNITS)': volume_to_allocate,
                        'PRICE': row['BID PRICE']
                    })
                    allocated_volume += volume_to_allocate
                    factory_demand_df.loc[factory_demand_df['PCX SPL MTL ID'] == factory['PCX SPL MTL ID'], 'VOLUME'] -= volume_to_allocate
    
    return pd.DataFrame(allocations)

# Perform the allocation
allocation_df = allocate_volume(merged_df, target_volume_df, factory_demand_df)

allocation_df.head()

In [ ]:
# Output the allocation results in the same format as ALLOCATION RESULT file
allocation_result_columns = ['PCX SPL MTL ID', 'SUPPLIER', 'FACTORY CODE', 'ALLOCATION (UNITS)', 'PRICE']
allocation_df = allocation_df[allocation_result_columns]

# Save the results to an Excel file
allocation_df.to_excel('ALLOCATION_RESULT.xlsx', index=False)

print("Allocation results saved to ALLOCATION_RESULT.xlsx")